In [1]:
from sklearn.cluster import KMeans

In [11]:
from scipy import io

all_vectors = io.mmread('/home/haonans/capstone/data/labeled_sparse_matrix.mtx').tocoo().todense()

print "All: {}".format(all_vectors.shape)

train_vectors, test_vectors = all_vectors[:15000], all_vectors[15000:]
print "Train: {}, Test: {}".format(train_vectors.shape, test_vectors.shape)

All: (16535, 1024)
Train: (15000, 1024), Test: (1535, 1024)


In [26]:
with open('../../data/labeled_testing_fnames.txt') as reader:
    all_vector_names = map(lambda line: line.strip().split('/')[-1], reader)
    
train_names, test_names = all_vector_names[:15000], all_vector_names[15000:]
assert len(train_names) == train_vectors.shape[0], len(test_names) == test_vectors.shape[0]

In [17]:
with open('../../data/labeled_testing/labels.txt') as reader:
    name_2_label = dict(map(lambda line: line.strip().split(), reader))
    
assert len(name_2_label) <= all_vectors.shape[0]

In [37]:
kmeans = KMeans(n_clusters=460, init='k-means++', n_jobs=-1)
kmeans.fit(train_vectors)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=460, n_init=10, n_jobs=-1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [34]:
kmeans = KMeans(n_clusters=230, init='k-means++', n_jobs=-1)
kmeans.fit(train_vectors)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=230, n_init=10, n_jobs=-1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [35]:
train_assignments = kmeans.predict(train_vectors)

from collections import defaultdict, Counter

cluster_2_list = defaultdict(list)
for i_name, i_cluster in enumerate(train_assignments):
    cluster_2_list[i_cluster].append(name_2_label.get(train_names[i_name], "XXX"))

cluster_2_label = {}
for i_cluster, labels in cluster_2_list.iteritems():
    counter = Counter(labels)
    cluster_2_label[i_cluster] = counter.most_common(1)[0][0]

In [36]:
correct_counter = 0

test_assignments = kmeans.predict(test_vectors)
for i_name, i_cluster in enumerate(test_assignments):
    real_label = name_2_label.get(test_names[i_name], "YYY")
    assigned_label = cluster_2_label[i_cluster]
    
    if real_label == assigned_label:
        correct_counter += 1
        
print "Total: {}, Correct: {}".format(len(test_names), correct_counter)

Total: 1535, Correct: 219


In [40]:
all_assignments = kmeans.predict(all_vectors)

from collections import defaultdict, Counter

cluster_2_list = defaultdict(list)
for i_name, i_cluster in enumerate(all_assignments):
    cluster_2_list[i_cluster].append(name_2_label.get(all_vector_names[i_name], "XXX"))

cluster_2_label = {}
for i_cluster, labels in cluster_2_list.iteritems():
    counter = Counter(labels)
    cluster_2_label[i_cluster] = counter.most_common(1)[0][0]

correct_counter = 0
for i_name, i_cluster in enumerate(all_assignments):
    real_label = name_2_label.get(all_vector_names[i_name], "YYY")
    assigned_label = cluster_2_label[i_cluster]
    
    if real_label == assigned_label:
        correct_counter += 1
        
print "Total: {}, Correct: {}".format(len(all_vector_names), correct_counter)

Total: 16535, Correct: 6085
